In [1]:
import os, json
import pandas as pd
import numpy as np
import glob
import re
from geopy.distance import geodesic
import dask.dataframe as dd
import datetime
import ooipy
import matplotlib.pyplot as plt
from datetime import timedelta
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

c:\Users\khirod\miniconda3\envs\mynlpenv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\khirod\miniconda3\envs\mynlpenv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\khirod\miniconda3\envs\mynlpenv\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
os.getcwd()

'd:\\drive D\\Acer backup 3\\internships and jobs\\UW\\Reader grader and TA\\Shima Abadi'

In [3]:
#ais = pd.read_csv('Axial Base-20220628T081651Z-001/Axial Base/2015A - Updated.csv',sep = ',')

### Append all the files

In [4]:
appended_data = []
for infile in glob.glob("Axial Base-20220628T081651Z-001/Axial Base/*.csv"):
    data = pd.read_csv(infile)
    numbers=re.findall('[0-9]+',infile)
    data['Year']=numbers[3]
    # store DataFrame in list
    appended_data.append(data)
# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)
ais=appended_data.copy()

In [5]:
ais.head()

,MMSI,SHIPNAME,VESSEL TYPE,STATUS,SPEED (KNOTSx10),LAT,LON,COURSE,HEADING,TIMESTAMP UTC,LENGTH,Year
0,209605000,AKILI,Bulk Carrier,0,108,46.09859,-129.6550,83,84,2015-01-01 00:28:00,189.99,2014
1,256832000,NaN,NaN,0,115,45.07486,-128.9430,178,181,2015-01-01 00:28:00,NaN,2014
2,352358000,ANNA G,Bulk Carrier,0,132,45.86138,-130.5627,86,85,2015-01-01 00:28:00,229.00,2014
3,356566000,GLOBAL SAIKAI,General Cargo,0,104,46.50261,-129.0129,272,275,2015-01-01 00:31:00,188.50,2014
4,477293500,JIN XIU FENG,Bulk Carrier,0,112,46.74633,-129.5912,299,298,2015-01-01 00:48:00,229.00,2014


In [6]:
ais['Year']=ais['Year'].astype('int')

In [7]:
ais.head()


,MMSI,SHIPNAME,VESSEL TYPE,STATUS,SPEED (KNOTSx10),LAT,LON,COURSE,HEADING,TIMESTAMP UTC,LENGTH,Year
0,209605000,AKILI,Bulk Carrier,0,108,46.09859,-129.6550,83,84,2015-01-01 00:28:00,189.99,2014
1,256832000,NaN,NaN,0,115,45.07486,-128.9430,178,181,2015-01-01 00:28:00,NaN,2014
2,352358000,ANNA G,Bulk Carrier,0,132,45.86138,-130.5627,86,85,2015-01-01 00:28:00,229.00,2014
3,356566000,GLOBAL SAIKAI,General Cargo,0,104,46.50261,-129.0129,272,275,2015-01-01 00:31:00,188.50,2014
4,477293500,JIN XIU FENG,Bulk Carrier,0,112,46.74633,-129.5912,299,298,2015-01-01 00:48:00,229.00,2014


In [8]:
ais['TIMESTAMP UTC'] = pd.to_datetime(ais['TIMESTAMP UTC']).dt.strftime('%Y-%m-%d %H:%M:%S')
ais['TIMESTAMP UTC'] = pd.to_datetime(ais['TIMESTAMP UTC'])

In [9]:
x=ais['TIMESTAMP UTC'].dt.day


### Calculate distance based on GPS coordinates

In [10]:
axial_Lat=45.8168
axial_Lon=-129.754
central_caldera_Lat = 45.9549
central_caldera_Lon=-130.0089
eastern_caldera_Lat = 45.9399
eastern_caldera_Lon = -129.9742



In [11]:
# For Axial base
ais['axial_Lat']=axial_Lat
ais['axial_Lon']=axial_Lon
#ais['axial_Loc']=pd.Series((axial_Lat,axial_Lon))
ais['axial_Loc'] = list(zip(ais.axial_Lat,ais.axial_Lon))
#ais['ship_Loc']=pd.Series((ais['LAT'],ais['LON']))
ais['ship_Loc'] = list(zip(ais.LAT,ais.LON))

#ais['ship_Loc'] = [(x,y) for x,y in zip(ais.LAT,ais.LON)] -> this could also work
ais.head()


,MMSI,SHIPNAME,VESSEL TYPE,STATUS,SPEED (KNOTSx10),LAT,LON,COURSE,HEADING,TIMESTAMP UTC,LENGTH,Year,axial_Lat,axial_Lon,axial_Loc,ship_Loc
0,209605000,AKILI,Bulk Carrier,0,108,46.09859,-129.6550,83,84,2015-01-01 00:28:00,189.99,2014,45.8168,-129.754,"(45.8168, -129.754)","(46.09859, -129.655)"
1,256832000,NaN,NaN,0,115,45.07486,-128.9430,178,181,2015-01-01 00:28:00,NaN,2014,45.8168,-129.754,"(45.8168, -129.754)","(45.07486, -128.943)"
2,352358000,ANNA G,Bulk Carrier,0,132,45.86138,-130.5627,86,85,2015-01-01 00:28:00,229.00,2014,45.8168,-129.754,"(45.8168, -129.754)","(45.86138, -130.5627)"
3,356566000,GLOBAL SAIKAI,General Cargo,0,104,46.50261,-129.0129,272,275,2015-01-01 00:31:00,188.50,2014,45.8168,-129.754,"(45.8168, -129.754)","(46.50261, -129.0129)"
4,477293500,JIN XIU FENG,Bulk Carrier,0,112,46.74633,-129.5912,299,298,2015-01-01 00:48:00,229.00,2014,45.8168,-129.754,"(45.8168, -129.754)","(46.74633, -129.5912)"


In [12]:
# For central caldera
ais['central_caldera_Lat']=central_caldera_Lat
ais['central_caldera_Lon']=central_caldera_Lon
#ais['axial_Loc']=pd.Series((axial_Lat,axial_Lon))
ais['central_caldera_Loc'] = list(zip(ais.central_caldera_Lat,ais.central_caldera_Lon))

In [13]:
# For eastern caldera
ais['eastern_caldera_Lat']=eastern_caldera_Lat
ais['eastern_caldera_Lon']=eastern_caldera_Lon
#ais['axial_Loc']=pd.Series((axial_Lat,axial_Lon))
ais['eastern_caldera_Loc'] = list(zip(ais.eastern_caldera_Lat,ais.eastern_caldera_Lon))

In [14]:
#Function to calculate distance distance from axial base
def geodistance_axial(row):
    coords_1 = row.axial_Loc
    coords_2 = row.ship_Loc
    return round(geodesic(coords_1, coords_2).km,2)


In [15]:
#Function to calculate distance distance from central caldera
def geodistance_central(row):
    coords_1 = row.central_caldera_Loc
    coords_2 = row.ship_Loc
    return round(geodesic(coords_1, coords_2).km,2)

In [16]:
#Function to calculate distance distance from eastern caldera
def geodistance_eastern(row):
    coords_1 = row.eastern_caldera_Loc
    coords_2 = row.ship_Loc
    return round(geodesic(coords_1, coords_2).km,2)

In [17]:
#dask_ais=dd.from_pandas(ais,npartitions=10)
#dask_ais.head()
#from dask.multiprocessing import get
#ais['distance']=geodesic(ais['axial_Loc'], ais['ship_Loc']).km
#x_out = dask_ais.map_partitions(lambda df:df.apply(geodistance,axis=1),meta=('result', int)).compute()#meta is important

In [18]:
ais['distance(in km) axial'] = ais.apply(geodistance_axial, axis=1)
ais['distance(in km) central cald'] = ais.apply(geodistance_central, axis=1)
ais['distance(in km) eastern cald'] = ais.apply(geodistance_eastern, axis=1)
ais.head()

,MMSI,SHIPNAME,VESSEL TYPE,STATUS,SPEED (KNOTSx10),LAT,LON,COURSE,HEADING,TIMESTAMP UTC,...,ship_Loc,central_caldera_Lat,central_caldera_Lon,central_caldera_Loc,eastern_caldera_Lat,eastern_caldera_Lon,eastern_caldera_Loc,distance(in km) axial,distance(in km) central cald,distance(in km) eastern cald
0,209605000,AKILI,Bulk Carrier,0,108,46.09859,-129.6550,83,84,2015-01-01 00:28:00,...,"(46.09859, -129.655)",45.9549,-130.0089,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",32.25,31.72,30.37
1,256832000,NaN,NaN,0,115,45.07486,-128.9430,178,181,2015-01-01 00:28:00,...,"(45.07486, -128.943)",45.9549,-130.0089,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",104.04,128.46,125.45
2,352358000,ANNA G,Bulk Carrier,0,132,45.86138,-130.5627,86,85,2015-01-01 00:28:00,...,"(45.86138, -130.5627)",45.9549,-130.0089,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",63.02,44.21,46.50
3,356566000,GLOBAL SAIKAI,General Cargo,0,104,46.50261,-129.0129,272,275,2015-01-01 00:31:00,...,"(46.50261, -129.0129)",45.9549,-130.0089,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",95.33,98.03,97.02
4,477293500,JIN XIU FENG,Bulk Carrier,0,112,46.74633,-129.5912,299,298,2015-01-01 00:48:00,...,"(46.74633, -129.5912)",45.9549,-130.0089,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",104.08,93.66,94.37


In [19]:
ais.columns

Index(['MMSI', 'SHIPNAME', 'VESSEL TYPE', 'STATUS', 'SPEED (KNOTSx10)', 'LAT',
       'LON', 'COURSE', 'HEADING', 'TIMESTAMP UTC', 'LENGTH', 'Year',
       'axial_Lat', 'axial_Lon', 'axial_Loc', 'ship_Loc',
       'central_caldera_Lat', 'central_caldera_Lon', 'central_caldera_Loc',
       'eastern_caldera_Lat', 'eastern_caldera_Lon', 'eastern_caldera_Loc',
       'distance(in km) axial', 'distance(in km) central cald',
       'distance(in km) eastern cald'],
      dtype='object')

In [2]:
ais = pd.read_csv('ais_2014B_2020B_V2.csv',sep = ',')

### Find the bearing angles

In [3]:
from math import degrees, atan2
import math
def calculate_initial_compass_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

In [4]:
ais['bearing_axial'] = [calculate_initial_compass_bearing(eval(x),eval(y)) for x,y in zip(ais['ship_Loc'],ais['axial_Loc'])]
ais['bearing_eastern_cald'] = [calculate_initial_compass_bearing(eval(x),eval(y)) for x,y in zip(ais['ship_Loc'],ais['eastern_caldera_Loc'])]
ais['bearing_central_cald'] = [calculate_initial_compass_bearing(eval(x),eval(y)) for x,y in zip(ais['ship_Loc'],ais['central_caldera_Loc'])]

### Export the dataframe to csv

In [5]:
ais.to_csv('ais_2014B_2020B_V2.csv')

In [6]:
ais.head()

,Unnamed: 0,MMSI,SHIPNAME,VESSEL TYPE,STATUS,SPEED (KNOTSx10),LAT,LON,COURSE,HEADING,...,central_caldera_Loc,eastern_caldera_Lat,eastern_caldera_Lon,eastern_caldera_Loc,distance(in km) axial,distance(in km) central cald,distance(in km) eastern cald,bearing_axial,bearing_eastern_cald,bearing_central_cald
0,0,209605000,AKILI,Bulk Carrier,0,108,46.09859,-129.6550,83,84,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",32.25,31.72,30.37,193.760689,234.515164,239.809636
1,1,256832000,NaN,NaN,0,115,45.07486,-128.9430,178,181,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",104.04,128.46,125.45,322.804182,320.490292,320.057435
2,2,352358000,ANNA G,Bulk Carrier,0,132,45.86138,-130.5627,86,85,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",63.02,44.21,46.50,94.234035,78.935325,76.159593
3,3,356566000,GLOBAL SAIKAI,General Cargo,0,104,46.50261,-129.0129,272,275,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",95.33,98.03,97.02,217.082385,230.114815,231.878577
4,4,477293500,JIN XIU FENG,Bulk Carrier,0,112,46.74633,-129.5912,299,298,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",104.08,93.66,94.37,186.960616,198.291220,200.167826
